In [17]:
import pickle, os, glob
import pandas as pd
from obspy import UTCDateTime

In [18]:
def ReadPKL_manual(evtpath):
    Meta_pkl = evtpath + '/Meta_manual.pkl'
    Station_data_pkl = evtpath + '/Station_data.pkl'
    Split_result_pkl = evtpath + '/Split_results_manual.pkl'
    SC_result_pkl = evtpath + '/SC_res_manual.pkl'
    RC_result_pkl = evtpath + '/RC_res_manual.pkl'
#     pick_pkl = evtpath + '/Pickphase.pkl'
    Meta = pickle.load(open(Meta_pkl, "rb"))
    Station_data = pickle.load(open(Station_data_pkl, "rb"))
    Split_result = pickle.load(open(Split_result_pkl, "rb"))
    SC_res = pickle.load(open(SC_result_pkl, "rb"))
    RC_res = pickle.load(open(RC_result_pkl, "rb"))
#     Pick = pickle.load(open(pick_pkl, "rb"))
    return Meta, Station_data, Split_result, SC_res, RC_res
def ReadPKL_auto(evtpath):
    Meta_pkl = evtpath + '/Meta_data.pkl'
    Station_data_pkl = evtpath + '/Station_data.pkl'
    Split_result_pkl = evtpath + '/Split_results_auto.pkl'
    SC_result_pkl = evtpath + '/SC_res.pkl'
    RC_result_pkl = evtpath + '/RC_res.pkl'
    Meta = pickle.load(open(Meta_pkl, "rb"))
    Station_data = pickle.load(open(Station_data_pkl, "rb"))
    Split_result = pickle.load(open(Split_result_pkl, "rb"))
    SC_res = pickle.load(open(SC_result_pkl, "rb"))
    RC_res = pickle.load(open(RC_result_pkl, "rb"))
    return Meta, Station_data, Split_result, SC_res, RC_res
def Creatdf():
    file = {'Event':[],'Ev_lat':[],'Ev_lon':[], 'Depth':[],'Mag':[],'Phase':[],
            'SCPhi':[],'SCPhi_std':[],'SCdt':[],'SCdt_std':[],
            'RCPhi':[],'RCPhi_std':[],'RCdt':[],'RCdt_std':[],
            'SNRQ':[],'SNRT':[],'Null':[],'Quality':[],'CpH':[],'Pick':[]}
    return file
def Append(df,ev_lat,ev_lon,dep,mag,phase,
           SCphi,SCphi_std,SCdt,SCdt_std, RCphi,RCphi_std,RCdt,RCdt_std,
           snrq,snrt,null,quality,CpH,pick=True):
    df['Ev_lat'].append(ev_lat)
    df['Ev_lon'].append(ev_lon)
    df['Depth'].append(dep)
    df['Mag'].append(mag)
    df['Phase'].append(phase)
    df['SCPhi'].append(SCphi)
    df['SCPhi_std'].append(SCphi_std)
    df['SCdt'].append(SCdt)
    df['SCdt_std'].append(SCdt_std)
    df['RCPhi'].append(RCphi)
    df['RCPhi_std'].append(RCphi_std)
    df['RCdt'].append(RCdt)
    df['RCdt_std'].append(RCdt_std)
    df['SNRQ'].append(snrq)
    df['SNRT'].append(snrt)
    df['Null'].append(null)
    df['Quality'].append(quality)
    df['CpH'].append(CpH)
    df['Pick'].append(pick)
    

In [10]:
DATApath = '/Volumes/home/Research/STEP/01_Split'
SAVEpath = '/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_Mw6_120-150degree_SKKS'

In [22]:
for STApath in sorted(glob.glob(f'{DATApath}/IU*.pkl')):
    STApkl = pickle.load(open(STApath, "rb"))
    values = [value for keys, value in STApkl.items() ][0]
    
    NET = values['network']; STA = values['station']
    print(STA)
    file = Creatdf()
    for Phase in ['SKKS']:
        for PHASEpath in sorted(glob.glob(f'{DATApath}/DATA_{Phase}_Mw6_ZRT/{NET}.{STA}')):
            for EVTpath in sorted(glob.glob(f'{PHASEpath}/*')):
                EVT = EVTpath.rsplit('/')[-1]
                if UTCDateTime(EVT.rsplit('_')[0])<=UTCDateTime(year=2021,month=1,day=1):
                        try: 
                            try :
                                Meta, Station_data, Split_result, SC_res, RC_res= ReadPKL_manual(EVTpath)
            #                     print('manual')
                                temp = Split_result.rsplit('/')[0]
                                NULL = temp.rsplit('? ')[-1]
                                temp = Split_result.rsplit('/')[1]
                                quality = temp.rsplit(': ')[-1]
                                Append(file,Meta.lat,Meta.lon,Meta.dep,Meta.mag,Meta.phase,
                                       round(SC_res.phi,2),round(SC_res.ephi,2),round(SC_res.dtt,2),round(SC_res.edtt,2),
                                       round(RC_res.phi,2),round(RC_res.ephi,2),round(RC_res.dtt,2),round(RC_res.edtt,2),
                                       round(Meta.snrq,2), round(Meta.snrt,2),NULL,quality,round(Meta.CpH,2),pick=True)
                            except:
                                Meta, Station_data, Split_result, SC_res, RC_res = ReadPKL_auto(EVTpath)
            #                     print('Auto')
                                temp = Split_result.rsplit('/')[0]
                                NULL = temp.rsplit('? ')[-1]
                                temp = Split_result.rsplit('/')[1]
                                quality = temp.rsplit(': ')[-1]
                                Append(file,Meta.lat,Meta.lon,Meta.dep,Meta.mag,Meta.phase,
                                       round(SC_res.phi,2),round(SC_res.ephi,2),round(SC_res.dtt,2),round(SC_res.edtt,2),
                                       round(RC_res.phi,2),round(RC_res.ephi,2),round(RC_res.dtt,2),round(RC_res.edtt,2),
                                       round(Meta.snrq,2),round(Meta.snrt,2),NULL,quality,round(Meta.CpH,2),pick=False) 
                            file['Event'].append(EVT)
                        except: print('============'+STA+EVT+'error ============')
    filename = f'{SAVEpath}/{NET}.{STA}_split_result_mw6_{Phase}.csv'
    df = pd.DataFrame(file)
    df.to_csv(filename,index=False)

GNI


In [16]:
df

,Event,Ev_lat,Ev_lon,Depth,Mag,Phase,SCPhi,SCPhi_std,SCdt,SCdt_std,RCPhi,RCPhi_std,RCdt,RCdt_std,SNRQ,SNRT,Null,Quality,CpH,Pick
0,20120124_005204,-24.8986,178.5999,579.0,6.3,SKKS,51.05,6.00,1.4,0.18,38.05,44.75,1.2,0.98,11.55,3.67,False,Fair,0.48,True
1,20120130_051059,-14.2139,-75.6584,34.2,6.4,SKKS,74.81,44.75,1.8,0.98,47.81,44.75,1.1,0.98,11.27,6.47,True,Poor,0.69,False
2,20120423_173621,-28.6015,-177.3408,113.8,6.0,SKKS,55.76,19.25,0.9,0.60,38.76,17.50,0.7,0.50,16.66,5.75,False,Poor,0.75,True
3,20120428_100807,-18.7293,-174.6864,132.5,6.7,SKKS,-26.55,44.75,3.9,0.98,17.45,44.75,0.0,0.98,12.16,3.70,True,Good,0.83,True
4,20120703_103615,-40.0600,173.6600,239.0,6.3,SKKS,5.78,12.25,3.3,0.88,-38.22,44.75,0.5,0.98,13.12,2.25,True,Good,0.93,True
5,20120706_022822,-14.6567,167.3043,164.8,6.4,SKKS,4.10,44.75,0.6,0.98,35.10,44.75,0.3,0.98,11.99,-0.32,True,Poor,0.89,True
6,20121114_190205,-29.1789,-71.3863,61.5,6.2,SKKS,23.39,44.75,1.1,0.98,34.39,44.75,1.0,0.98,0.74,4.04,False,Fair,0.61,True
7,20121221_222808,-14.3591,167.2778,198.1,6.7,SKKS,-4.16,44.75,1.0,0.98,36.84,44.75,0.3,0.98,8.80,-3.57,True,Poor,0.88,True
8,20130605_044727,-11.4032,166.3017,53.6,6.0,SKKS,-3.05,1.25,3.3,0.35,42.95,44.75,0.5,0.98,1.09,4.64,True,Good,0.85,True
9,20130724_033234,-23.1593,-177.1534,168.0,6.0,SKKS,-21.82,44.75,3.9,0.98,20.18,44.75,0.1,0.98,8.62,1.00,True,Good,0.83,True
